## Lista 7 - Exercício de Mineração de texto

#### Questão 1 

Use o dataset "tweets_trump.csv", que contém todos os tweets do D. Trump e faça o que é pedido abaixo.

a) Utilizando o texto dos tweets identifique 5 tópicos latentes com o LDA, com um preprocessamento básico -- remoção de urls, remoção de pontuação, stemming e conversão para minúsculo. 

b) Avalie os tópicos encontrados. Algum preprocessamento a mais poderia ser útil para a produção de um melhor resultado?


In [22]:
%pip install nltk
%pip install spacy
%pip install pyLDAvis


   ---------------------------------------- 1.5/1.5 MB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 12.2/12.2 MB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 182.0/182.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 122.7/122.7 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 57.0/57.0 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 483.8/483.8 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 1.5/1.5 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 6.6/6.6 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 45.0/45.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 5.4/5.4 MB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 152.8/152.8 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: colorama
    Found existing installati


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 2.6/2.6 MB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 15.8/15.8 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 24.0/24.0 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [50]:
import pandas as pd
import numpy as np
import string
import re, nltk, spacy, gensim # Sklearn
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
df = pd.read_csv('kaggle/input/trump_tweets/tweets_trump.csv')
df['text'].head()

nltk.download('stopwords')
stop_words = stopwords.words('english')


def remove_rt(text):
    return re.sub('RT @\w+: ', "", text)

# Function to remove URLs from text
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove punctuation from text
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    words = [w for w in text.split() if w not in stop_words]
    return words


# Apply the preprocessing functions to the 'text' column
df['text'] = df['text'].apply(remove_rt)
df['text'] = df['text'].apply(remove_urls)
df['text'] = df['text'].apply(remove_punctuation)
df['text'] = df['text'].str.lower()


data = df.text.values.tolist()
data = [remove_stopwords(text) for text in data]

print(data[:5])

<>:9: DeprecationWarning: invalid escape sequence \w
<>:9: DeprecationWarning: invalid escape sequence \w
C:\Users\danie\AppData\Local\Temp\ipykernel_21464\1419421169.py:9: DeprecationWarning: invalid escape sequence \w
  return re.sub('RT @\w+: ', "", text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['republicans', 'democrats', 'created', 'economic', 'problems'], ['thrilled', 'back', 'great', 'city', 'charlotte', 'north', 'carolina', 'thousands', 'hardworking', 'american', 'patriots', 'love', 'country', 'cherish', 'values', 'respect', 'laws', 'always', 'put', 'america', 'first', 'thank', 'wonderful', 'evening', 'kag2020'], ['read', 'letter', 'surveillance', 'court', 'obtained', 'cbs', 'news', 'questions', 'disciplinary', 'action', 'cho…'], ['unsolicited', 'mail', 'ballot', 'scam', 'major', 'threat', 'democracy', 'amp', 'democrats', 'know', 'almost', 'recent', 'elections', 'using', 'system', 'even', 'though', 'much', 'smaller', 'amp', 'far', 'fewer', 'ballots', 'count', 'ended', 'disaster', 'large', 'numbers', 'missing', 'ballots', 'amp', 'fraud'], ['friendly', 'telling', 'events', 'comeys', 'apparent', 'leaking', 'compliant', 'media', 'read', 'articles', 'tho…']]


In [20]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))
pprint(data_words[:5])

[['republicans', 'democrats', 'created', 'economic', 'problems'],
 ['thrilled',
  'back',
  'great',
  'city',
  'charlotte',
  'north',
  'carolina',
  'thousands',
  'hardworking',
  'american',
  'patriots',
  'love',
  'country',
  'cherish',
  'values',
  'respect',
  'laws',
  'always',
  'put',
  'america',
  'first',
  'thank',
  'wonderful',
  'evening',
  'kag'],
 ['read',
  'letter',
  'surveillance',
  'court',
  'obtained',
  'cbs',
  'news',
  'questions',
  'disciplinary',
  'action',
  'cho'],
 ['unsolicited',
  'mail',
  'ballot',
  'scam',
  'major',
  'threat',
  'democracy',
  'amp',
  'democrats',
  'know',
  'almost',
  'recent',
  'elections',
  'using',
  'system',
  'even',
  'though',
  'much',
  'smaller',
  'amp',
  'far',
  'fewer',
  'ballots',
  'count',
  'ended',
  'disaster',
  'large',
  'numbers',
  'missing',
  'ballots',
  'amp',
  'fraud'],
 ['friendly',
  'telling',
  'events',
  'comeys',
  'apparent',
  'leaking',
  'compliant',
  'media',
  'r

In [27]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out



In [28]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_words = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) 
data_words[:5]

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models and if you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [43]:
data_words[:5]

['create economic problem',
 'thrill back great city thousand hardworke american patriot love country cherish value respect law always put first thank wonderful evening kag',
 'read letter surveillance court obtain news question disciplinary action cho',
 'unsolicite mail ballot scam major threat democracy know almost recent election use system even much small amp far few ballot count end disaster large number miss ballot amp fraud',
 'friendly tell event comey apparent leak compliant medium article tho']

In [42]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                              max_features=50000,             )
data_vectorized = vectorizer.fit_transform(data_words)

print(data_vectorized)

  (0, 748)	1
  (0, 1048)	1
  (0, 2554)	1
  (1, 3329)	1
  (1, 250)	1
  (1, 1469)	1
  (1, 547)	1
  (1, 3325)	1
  (1, 1515)	1
  (1, 124)	1
  (1, 2365)	1
  (1, 1962)	1
  (1, 727)	1
  (1, 528)	1
  (1, 3546)	1
  (1, 2777)	1
  (1, 1870)	1
  (1, 113)	1
  (1, 2616)	1
  (1, 1296)	1
  (1, 3299)	1
  (1, 3686)	1
  (1, 1157)	1
  (1, 1818)	1
  (2, 2664)	1
  :	:
  (56567, 931)	1
  (56568, 823)	2
  (56568, 3591)	1
  (56568, 2525)	1
  (56568, 1934)	1
  (56568, 3298)	1
  (56569, 124)	3
  (56569, 113)	1
  (56569, 2616)	1
  (56569, 1296)	1
  (56569, 2588)	1
  (56569, 1995)	1
  (56569, 3215)	1
  (56569, 3209)	1
  (56569, 3690)	5
  (56569, 2288)	1
  (56569, 3008)	1
  (56569, 1223)	2
  (56569, 713)	1
  (56570, 3606)	1
  (56570, 2869)	1
  (56570, 3683)	1
  (56570, 1434)	1
  (56570, 2929)	1
  (56570, 3186)	1


In [38]:
lda_model = LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method='online', learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=5, n_jobs=-1, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

lda_output = lda_model.fit_transform(data_vectorized)

In [39]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2931093.843290576
Perplexity:  1443.6077514931105
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 5,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [51]:
# Create Document — Topic Matrix
lda_output = lda_model.transform(data_vectorized)# column names
topicnames = ['Topic' + str(i) for i in range(lda_model.n_components)]# index names
docnames = ['Doc' + str(i) for i in range(len(data))]# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics



C:\Users\danie\AppData\Local\Temp\ipykernel_21464\1269774312.py:14: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)


,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.050000,0.050000,0.050000,0.050000,0.800000,4
Doc1,0.140000,0.010000,0.010000,0.790000,0.050000,3
Doc2,0.690000,0.130000,0.020000,0.130000,0.020000,0
Doc3,0.010000,0.010000,0.250000,0.730000,0.010000,3
Doc4,0.030000,0.030000,0.030000,0.900000,0.030000,3
Doc5,0.820000,0.120000,0.020000,0.020000,0.020000,0
Doc6,0.040000,0.040000,0.160000,0.720000,0.040000,3
Doc7,0.200000,0.200000,0.200000,0.200000,0.200000,0
Doc8,0.200000,0.200000,0.200000,0.200000,0.200000,0
Doc9,0.200000,0.200000,0.200000,0.200000,0.200000,0


In [54]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names_out()
df_topic_keywords.index = topicnames# View
df_topic_keywords.head()

,abandon,aberdeen,aberdeenshire,ability,able,abolish,abortion,absentee,absolute,absolutely,...,year,yearly,yell,yesterday,yet,young,youth,youtube,yrs,zone
Topic0,0.205733,0.201779,0.202053,0.202419,61.396126,0.205467,13.950429,0.200494,0.204350,189.162315,...,0.206718,0.204780,0.200825,0.204247,3.288409,0.204266,0.218053,0.202620,0.204609,0.202741
Topic1,33.496948,0.204027,0.201768,0.206226,0.206138,31.000032,0.204815,0.200698,0.204487,0.205118,...,1002.288713,0.204749,8.360062,450.718910,7.669560,0.204423,0.204720,13.986065,0.203087,0.202043
Topic2,0.204126,0.203269,0.202943,0.203381,141.595638,0.202266,0.202414,26.917049,60.517739,0.205853,...,1256.111777,0.202394,0.201445,0.205282,0.205315,0.202101,20.095576,0.207948,0.203849,0.202028
Topic3,0.207199,0.202392,0.200523,0.204977,0.206519,0.201801,0.202276,0.200807,0.205753,0.206344,...,0.206351,0.203045,0.201754,0.206369,127.787660,0.203026,0.208391,0.202225,0.201677,0.201470
Topic4,0.206171,22.298175,8.806585,41.203826,0.208902,0.204321,0.202470,0.200302,0.203170,47.579606,...,0.209091,11.707221,0.202593,0.205504,154.308501,143.291485,0.207629,0.201928,15.891660,57.513947


In [58]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,realdonaldtrump,job,president,trump,border,think,love,run,work,strong,country,long,hard,people,military
Topic 1,great,make,realdonaldtrump,want,come,people,day,year,see,get,look,true,state,start,leave
Topic 2,trump,get,vote,today,need,new,bad,say,year,election,call,country,good,time,win
Topic 3,thank,great,never,many,good,news,know,amp,fake,even,show,medium,much,tonight,interview
Topic 4,well,take,deal,world,high,talk,record,big,lose,economy,keep,trade,way,time,soon


Os tópicos parecem ser:
Topic 0: Nacionalismo americano
Topic 2: Campanha e slogan
Topic 3: Eleições e críticas aos oponentes
Topic 4: Notícias e entrevistas
Topic 5: Economia

O resultado poderia ser melhor se tivessemos uma lista de palavras stop que abranja erros de digitação e palavras que não são relevantes para a análise. Além disso, a remoção de palavras comuns como "great" e "thank" poderia melhorar a qualidade dos tópicos.

## Questão 2

Use o dataset "tweets_trump.csv" e responda:

a) Qual o sentimento dos 5 tweets com maior número de retweets?

b) Faça uma análise do sentimento geral de todos os tweets. Para isso, visualize a distribuição de polaridade para embasar a sua resposta.

In [60]:
from transformers import pipeline #hugging face pipeline
import os

df = pd.read_csv('kaggle/input/trump_tweets/tweets_trump.csv')

df_sorted = df.sort_values('retweets', ascending=False)
df_sorted['text'].head()

11759    Tonight, @FLOTUS and I tested positive for COV...
35620           #FraudNewsCNN #FNN https://t.co/WYUnHjjUjg
39347                   TODAY WE MAKE AMERICA GREAT AGAIN!
29598    Are you allowed to impeach a president for gro...
9080          RT @SpaceX: Liftoff! https://t.co/DRBfdUM7JA
Name: text, dtype: object

Sem realizar processamento podemos ver que o sentimento dos mais retwitados é majoritariamente positivo.
Sem contexto  